# Data and Library Importing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import confusion_matrix,classification_report,f1_score

import time
import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from optuna.visualization import plot_pareto_front

from tensorflow.keras import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input,Dense,concatenate,Dropout,Conv1D,Bidirectional,Embedding,LSTM,SimpleRNN,GlobalAveragePooling1D
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import CategoricalCrossentropy
from tensorflow.random import set_seed

In [2]:
set_seed(123)
np.random.seed(123)

In [3]:
label_latih=np.load('../input/emot-dataset/label_latih.npy', allow_pickle=True)
label_validasi=np.load('../input/emot-dataset/label_validasi.npy', allow_pickle=True)
label_uji=np.load('../input/emot-dataset/label_uji.npy', allow_pickle=True)

In [4]:
embedding_dim = 450
max_length = 80
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

In [5]:
teks_latih_pad=np.load('../input/emot-dataset/teks_latih_pad.npy', allow_pickle=True)
teks_latih=np.load('../input/emot-dataset/teks_latih.npy', allow_pickle=True)
teks_validasi_pad=np.load('../input/emot-dataset/teks_validasi_pad.npy', allow_pickle=True)
teks_validasi=np.load('../input/emot-dataset/teks_validasi.npy', allow_pickle=True)
teks_uji_pad=np.load('../input/emot-dataset/teks_uji_pad.npy', allow_pickle=True)
teks_uji=np.load('../input/emot-dataset/teks_uji.npy', allow_pickle=True)

# Modelling

In [6]:
jumlah_token=17039

In [7]:
def create_model(trial):
  lstm_units=np.zeros(2, dtype=int)
  lstm_units[0]=trial.suggest_int("lstm_units_L1", 16, 48)
  lstm_units[1]=trial.suggest_int("lstm_units_L2", 4, 20)
  do1=trial.suggest_float("dropout_L1",0,0.5)
  do2=trial.suggest_float("dropout_L2",0,0.5)
  lr=trial.suggest_float("learning_rate",1e-5,1e-2)

  inputs=Input(shape=(max_length,))
  layers=Embedding(jumlah_token, embedding_dim,input_length=max_length,trainable=False)(inputs)
  layers=Bidirectional(LSTM(lstm_units[0], return_sequences=True))(layers)
  layers=Dropout(do1)(layers)
  layers=Bidirectional(LSTM(lstm_units[1]))(layers)
  layers=Dropout(do2)(layers)
  layers=Dense(8, activation='relu')(layers)
  output=Dense(5, activation='softmax')(layers)

  model=Model(inputs=[inputs],outputs=[output])

  model.compile(optimizer=Adam(learning_rate=lr),loss="sparse_categorical_crossentropy",metrics=["accuracy"])

  return model

## Bayesian Optimization (F1-score)

In [8]:
#Fungsi Objektif
def objective_fun_f1(trial):
  model=create_model(trial)

  start=time.time()
  history=model.fit(teks_latih_pad,label_latih,epochs=25,batch_size=16,callbacks=[TFKerasPruningCallback(trial, "val_loss")],
                    validation_data=(teks_validasi_pad,label_validasi),verbose=0)
  end=time.time()

  Label_pred=model.predict(teks_uji_pad)
  label_pred=Label_pred.argmax(axis=1)
  fscore=f1_score(label_uji, label_pred, average='macro')
  duration=end-start
  model.save('Bayesian Optimization (F1-score) Model-{}.h5'.format(trial.number))

  print("F1-Score:",fscore)
  print("Duration:",duration)

  return fscore

In [9]:
study3=optuna.create_study(study_name="BO (F1-score)",
                          direction="maximize", 
                          sampler=optuna.samplers.TPESampler(), 
                          pruner=optuna.pruners.NopPruner())
study3.optimize(objective_fun_f1,n_trials=25)

[I 2023-03-09 13:46:44,417] A new study created in memory with name: BO (F1-score)
[I 2023-03-09 13:48:33,053] Trial 0 finished with value: 0.47342114612905684 and parameters: {'lstm_units_L1': 36, 'lstm_units_L2': 12, 'dropout_L1': 0.2473594948079108, 'dropout_L2': 0.36065902385639564, 'learning_rate': 0.0010598554484198056}. Best is trial 0 with value: 0.47342114612905684.


F1-Score: 0.47342114612905684
Duration: 102.45430183410645


[I 2023-03-09 13:50:15,546] Trial 1 finished with value: 0.476565128689292 and parameters: {'lstm_units_L1': 46, 'lstm_units_L2': 17, 'dropout_L1': 0.24834636529918147, 'dropout_L2': 0.08795974588372463, 'learning_rate': 0.003927032072488497}. Best is trial 1 with value: 0.476565128689292.


F1-Score: 0.476565128689292
Duration: 100.0890200138092


[I 2023-03-09 13:51:57,774] Trial 2 finished with value: 0.49383973621769134 and parameters: {'lstm_units_L1': 43, 'lstm_units_L2': 11, 'dropout_L1': 0.316439996936573, 'dropout_L2': 0.1285817805551775, 'learning_rate': 0.004295152271591475}. Best is trial 2 with value: 0.49383973621769134.


...

F1-Score: 0.4984247853017334
Duration: 96.40925168991089


[I 2023-03-09 14:26:58,559] Trial 23 finished with value: 0.5048609264654231 and parameters: {'lstm_units_L1': 16, 'lstm_units_L2': 16, 'dropout_L1': 0.41258931213743244, 'dropout_L2': 0.4439499236277576, 'learning_rate': 0.006612084219981152}. Best is trial 8 with value: 0.5428091367936703.


F1-Score: 0.5048609264654231
Duration: 95.86178994178772


[I 2023-03-09 14:28:38,919] Trial 24 finished with value: 0.502392719553504 and parameters: {'lstm_units_L1': 23, 'lstm_units_L2': 19, 'dropout_L1': 0.2995747898095854, 'dropout_L2': 0.15248494034070081, 'learning_rate': 0.008310129336805172}. Best is trial 8 with value: 0.5428091367936703.


F1-Score: 0.502392719553504
Duration: 97.24480128288269


In [10]:
plot_optimization_history(study3)

In [11]:
plot_intermediate_values(study3)

In [12]:
plot_param_importances(study3)

In [13]:
print("Best trial:")
trial = study3.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.5428091367936703
  Params: 
    lstm_units_L1: 47
    lstm_units_L2: 20
    dropout_L1: 0.19975302809066547
    dropout_L2: 0.3229935057718496
    learning_rate: 0.005419774867222899


## Bayesian Optimization (Time)

In [14]:
#Fungsi Objektif
def objective_fun_time(trial):
  model=create_model(trial)

  start=time.time()
  history=model.fit(teks_latih_pad,label_latih,epochs=25,batch_size=16,callbacks=[TFKerasPruningCallback(trial, "val_loss")],
                    validation_data=(teks_validasi_pad,label_validasi),verbose=0)
  end=time.time()

  Label_pred=model.predict(teks_uji_pad)
  label_pred=Label_pred.argmax(axis=1)
  fscore=f1_score(label_uji, label_pred, average='macro')
  duration=end-start
  model.save('Bayesian Optimization (Model Training Time) Model-{}.h5'.format(trial.number))

  print("F1-Score:",fscore)
  print("Duration:",duration)

  return duration

In [15]:
study4=optuna.create_study(study_name="BO (Time)",
                          direction="minimize", 
                          sampler=optuna.samplers.TPESampler(), 
                          pruner=optuna.pruners.NopPruner())
study4.optimize(objective_fun_time,n_trials=25)

[I 2023-03-09 14:28:39,724] A new study created in memory with name: BO (Time)
[I 2023-03-09 14:30:20,204] Trial 0 finished with value: 97.85267114639282 and parameters: {'lstm_units_L1': 41, 'lstm_units_L2': 18, 'dropout_L1': 0.47049506302732264, 'dropout_L2': 0.07577547799773449, 'learning_rate': 0.0054099711427083716}. Best is trial 0 with value: 97.85267114639282.


F1-Score: 0.49851404721404774
Duration: 97.85267114639282


[I 2023-03-09 14:32:00,259] Trial 1 finished with value: 97.96082854270935 and parameters: {'lstm_units_L1': 24, 'lstm_units_L2': 9, 'dropout_L1': 0.14255665776111848, 'dropout_L2': 0.30173101559532967, 'learning_rate': 0.00442208452609011}. Best is trial 0 with value: 97.85267114639282.


F1-Score: 0.5211212133000235
Duration: 97.96082854270935


[I 2023-03-09 14:33:40,710] Trial 2 finished with value: 97.40197277069092 and parameters: {'lstm_units_L1': 28, 'lstm_units_L2': 8, 'dropout_L1': 0.3206277951394539, 'dropout_L2': 0.46655633160363014, 'learning_rate': 0.004374022103632051}. Best is trial 2 with value: 97.40197277069092.


...

F1-Score: 0.46069379287779827
Duration: 94.7835419178009


[I 2023-03-09 15:08:13,437] Trial 23 finished with value: 98.49759006500244 and parameters: {'lstm_units_L1': 43, 'lstm_units_L2': 6, 'dropout_L1': 0.40299238485461275, 'dropout_L2': 0.35825481637162343, 'learning_rate': 0.0023592258240117334}. Best is trial 10 with value: 93.73201727867126.


F1-Score: 0.39838662818878984
Duration: 98.49759006500244


[I 2023-03-09 15:09:53,001] Trial 24 finished with value: 96.9167869091034 and parameters: {'lstm_units_L1': 40, 'lstm_units_L2': 8, 'dropout_L1': 0.49991235998314604, 'dropout_L2': 0.4234981398388217, 'learning_rate': 0.003431642013962826}. Best is trial 10 with value: 93.73201727867126.


F1-Score: 0.5155189073417268
Duration: 96.9167869091034


In [16]:
plot_optimization_history(study4)

In [17]:
plot_intermediate_values(study4)

In [18]:
plot_param_importances(study4)

In [19]:
print("Best trial:")
trial = study4.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  93.73201727867126
  Params: 
    lstm_units_L1: 48
    lstm_units_L2: 4
    dropout_L1: 0.4888417323221161
    dropout_L2: 0.37402524894190203
    learning_rate: 0.009496382415761381
